In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from azure.cosmos import CosmosClient

import os
from dotenv import load_dotenv
load_dotenv()

OPEN_AI_ENDPOINT = os.getenv("OPEN_AI_ENDPOINT")
subscription_key = os.getenv("SUBSCRIPTION_KEY")


In [11]:
llm = AzureChatOpenAI(
    api_version="2025-01-01-preview",
    azure_endpoint=OPEN_AI_ENDPOINT,
    api_key=subscription_key,
    temperature=0.1,     
)

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Professor. You are to answer the question in 20-30 words "),
    ("user", "tell me a about {topic}"),
])

chain = prompt | llm

response = chain.invoke({"topic": "Vector Databases"})
print(response.content)

Vector databases store and manage high-dimensional vector embeddings, enabling efficient similarity search and retrieval for applications like recommendation systems, natural language processing, and computer vision.


In [13]:
#embedding model text-embeddding-ada-002
EMBEDDING_MODEL_ENDPOINT = os.getenv("EMBEDDING_MODEL_ENDPOINT")

embedding = AzureOpenAIEmbeddings(
    azure_endpoint=EMBEDDING_MODEL_ENDPOINT,
    api_key=subscription_key,
)

In [14]:
query = "What is Azure OpenAI?"
query_embedding = embedding.embed_query(query)

#print the dimension of the embedding
print(len(query_embedding))

1536


In [15]:
DATABASE_NAME = os.getenv("DATABASE_NAME")
CONTAINER_NAME = os.getenv("CONTAINER_NAME")

In [16]:
COSMOS_CONNECTION_STRING = os.getenv("COSMOS_CONNECTION_STRING")

# Cosmos_client connnection using Connection string 
client = CosmosClient.from_connection_string(COSMOS_CONNECTION_STRING)
database = client.get_database_client(DATABASE_NAME)
container = database.get_container_client(CONTAINER_NAME)

In [17]:
def insert_document(test_id, content):
    content_vector = embedding.embed_query(content)
    document = {
        "id": str(test_id),
        "content": content,
        "content_vector": content_vector
    }
    container.create_item(body=document)
    print(f"Inserted document with test_id: {test_id}")


In [18]:
doc1 = insert_document(
    test_id="test_002",
    content="Azure OpenAI Service provides REST API access to OpenAI's powerful language models, enabling developers to integrate AI capabilities into their applications.",
)

Inserted document with test_id: test_002
